In [ ]:
import pandas as pd
import numpy as np
import folium

Download data from:  
<https://ffiec.cfpb.gov/data-publication/modified-lar/2021>

Make sure 2021 is selected and enter `B4TYDEB6GKMZO031MB27` as the LEI for Bank of America. Then click on "Download Modified LAR".

In [ ]:
df = pd.read_csv('B4TYDEB6GKMZO031MB27.csv')
print(len(df))
df.head()

In [ ]:
df = df[['action_taken', 'census_tract']]
df = df.dropna()
df['census_tract'] = df['census_tract'].astype(int)
print(df.dtypes)
df['census_tract'] = df['census_tract'].astype(str)


df = df[(df['action_taken'] == 1) | (df['action_taken'] == 3)]
print(df['action_taken'].value_counts())
print(df['census_tract'].nunique())

print(len(df))


In [ ]:
## subsample
# df = df.sample(n=120000)

## Retrieve Locations of Census Tracts from Census Gazetteer

Downloaded data from:  
<https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.2021.html>

Find `Census Tracts` and click on "Download the National Census Tracts Gazetteer Files".

In [ ]:
loc_df = pd.read_csv('2021_Gaz_tracts_national.csv', delimiter='\t')
print(loc_df.dtypes)
# loc_df.head()
loc_df.rename(columns={ loc_df.columns[-1]: "INTPTLONG" }, inplace = True)
# loc_df.columns


In [ ]:
tract2loc = dict(zip(loc_df['GEOID'], zip(loc_df['INTPTLAT'], loc_df['INTPTLONG'])))
# print(tract2loc[list(tract2loc.keys())[0]])



In [ ]:
# print(df.head(1))
# print(df.dtypes)

df['location'] = df['census_tract'].astype(int).map(tract2loc)

df[['lat', 'lon']] = pd.DataFrame(df['location'].tolist(), index=df.index)

# print(df.iloc[0])

df = df.dropna()
print(len(df))
print(df['action_taken'].value_counts())
df.head()


## Display

In [ ]:
df.to_csv('../LAR.csv')

In [ ]:
mapit = folium.Map(location=[37.09, -95.71], zoom_start=5, tiles="Stamen Toner")

for index, row in df.iterrows():
    if row['action_taken'] == 1:
        folium.CircleMarker( location=row['location'], color='#00FF00', fill_color='#00FF00', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )
    elif row['action_taken'] == 3:
        folium.CircleMarker( location=row['location'], color='#FF0000', fill_color='#FF0000', fill=True, opacity=0.4, fill_opacity=0.4, radius=2 ).add_to( mapit )

mapit
